<a href="https://colab.research.google.com/github/Steve-YJ/Extracting-Important-Feature-of-Malimg-using-VAE/blob/master/03_Training_MNIST_AAE_Add_loss_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. Training MNIST_AAE

1st. Run AAE Architecture on MNIST dataset<br>
2nd. Add Loss Plot<br> 
3rd. be familiar with Deep Learning Process<br>

## Reference
* god....!: https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/aae/aae.py

### What I want to do
from 20.07.15.wed<br>
<br>

* run AAE -> Done
* practicing train -> Do more
* practice & make my own work flow -> Make Template<br>
    * continue: https://github.com/Steve-YJ/Colab_Exercise/blob/master/Again_Training_Exp05_Just_20Epochs.ipynb

---
### Log
1. 20.07.15. Train 200 Epochs <- Original
2. 20.07.16. Change Parameters & Train 200 Epochs => Good
3. 20.07.16. Add module's 
    * Loss Graph
        * Generator, Discriminator and ELBO Loss
        
<br>
<br>
<br>

Reference<br>
* Add
    * Loss plot: Generator, Discriminator ...
    * save plot every 10 Epochs
    * plot latent vector Every Epoch
        * save it every Epoch
    * save model's state_dict per 10Epoch
    * save latent vector per 10Epoch
    

---
Start it<br><br><br><br><br>

## 00. Mount Drive

* If you use colab, first mount driver

First, always check if your GPU is possible or not


In [1]:
! nvidia-smi

Fri Jul 17 05:51:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

set 'autoreload'

In [2]:
%load_ext autoreload
%autoreload 2


Mount Drive

In [3]:
# mount drive(Drive Mount)

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
! pwd

/content


In [5]:
# %cd drive/My\ Drive/Post_InfoSec_Exp

% cd '/content/drive/My Drive/Post_InfoSec_Exp_200715'
! ls

/content/drive/My Drive/Post_InfoSec_Exp_200715
'01. MNIST_AAE.ipynb의 사본'				  data		 images
'02. Training MNIST_AAE_Parameter Tunning.ipynb의 사본'   Exp02_images
'03. Training MNIST_AAE_Add_loss_module.ipynb'		  Exp03_images


## 01. Library Import

In [6]:
import argparse
import os
import numpy as np
import math
import itertools

from torchvision import datasets
from torchvision.utils import save_image
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

# 데이터 전처리는 주로 torchvision에서 하는데?
# e.g torchivision.transforms, torchivision.save_image, torchivision.datasets

# 직접적으로 train과 관련된 부분은 torch를 활용한다
# torch.utils.data DataLoader, torch.autograd Variable, torch, torch.nn, torch.nn.functional

In [7]:
# os library를 사용해 directory를 만드는 방법을 배워보자

os.makedirs("Exp03_images", exist_ok=True)

In [8]:
! ls  # Exp03_images directory가 생성된 것을 확인할 수 있다

'01. MNIST_AAE.ipynb의 사본'				  data		 images
'02. Training MNIST_AAE_Parameter Tunning.ipynb의 사본'   Exp02_images
'03. Training MNIST_AAE_Add_loss_module.ipynb'		  Exp03_images


In [9]:
# Image shape를 정하고 간다
# 1channel 32 width 32 height

img_shape = (1, 32, 32)

cuda = True if torch.cuda.is_available() else False

Q. 모델의 파라미터를 어디에서 정해주면 좋을까?<br>

A1. Library Import 단계에서?<br>
A2. Training을 시작하기 전?(<- 후자가 좀 더 낫지 않을까?)

## 02. Data Preprocessing
* Load dataset
* preprocess it
    * transforms
    * make custom dataset
    * train_test split
    * make train loader, test loader

* work Flow
    * transforms module 사용해서 image data compose
        * size 조정 및 normalize, tensor 변환
        * Image Folder를 이용해 dataload
        * dataset split: train dataset, test dataset
        * DataLoader로 batch단위 dataset 불러오기

### 2-1. Configure data loader
* 본 Tutorial에서는 Data Loader에 대해서만 신경써주면 될 것 같다
<br>
<br>


Parameter settings<br>
* for preprocessing
    * transforms.Resize: 32
    * transforms.Normalize: 0.5(mean), 0.5(std)
* for Data Loader
    * batch_size: 64
    * shuffle: True(shuffle it)


In [10]:
# configure data loader

os.makedirs("./data/mnist", exist_ok=True)  # make directory: ./data/mnist

# Define DataLoader: Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset. 
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",  # root
        train=True,
        download=True,
        transform = transforms.Compose(
            [transforms.Resize(32),  # Resizing
             transforms.ToTensor(),  # convert numpy to tensor
             transforms.Normalize([0.5], [0.5])]  # transforms.Normalize(): Normalize a tensor image with mean and standard deviation.
        ),
    ),
    batch_size=64,
    shuffle=True
)  

# 03. Define Adversarial Variational AutoEncoder Class

* Encoder Class
* Decoder Class
* Discriminator Class
* Optimizer

### reparameterization module

In [22]:
def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), 10))))  # np.random.normal(): Draw random samples from a normal (Gaussian) distribution.
                                                                            # mean:0, std: 1
                                                                            # (batch_size, 10)
    z = sampled_z * std + mu  # latent vector z
    return z

### 3-1. Encoder Class

In [12]:
class Encoder(nn.Module):
    def __init__(self):  # initialization
        super(Encoder, self).__init__()

        self.model = nn.Sequential(  # nn.Sequential: A sequential container. Modules will be added to it in the order they are passed in the constructor. Alternatively, an ordered dict of modules can also be passed in.
            nn.Linear(int(np.prod(img_shape)), 512),  # Linear Transform
            nn.LeakyReLU(0.2, inplace=True),  # Activation Function
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )  
        
        self.mu = nn.Linear(512, 10)      # 10: latent dim
        self.logvar = nn.Linear(512, 10)

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)  # 1 x (32*32)
        x = self.model(img_flat)  # extract_vector: (batch_size, 10)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = reparameterization(mu, logvar)
        return z


In [13]:
np.prod(img_shape)

1024

### 3-2. Decoder

In [14]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, int(np.prod(img_shape))),  # (batch_size, 1024)
            nn.Tanh(),
        )

    def forward(self, z):
        img_flat = self.model(z)
        img = img_flat.view(img_flat.shape[0], *img_shape)
        return img

### 3-3. Discriminator

In [15]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, z):
        validity = self.model(z)
        return validity

### Define Loss Function

In [16]:
# Use binary cross-entropy loss

adversarial_loss = torch.nn.BCELoss()
pixelwise_loss = torch.nn.L1Loss()

### Initialize Generator & Discriminator



In [17]:
# Initialize generator and discriminator
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()

if cuda:
    encoder.cuda()
    decoder.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    pixelwise_loss.cuda()

In [18]:
discriminator.parameters()

<generator object Module.parameters at 0x7f2fadb6b3b8>

### Optimizer

In [19]:
optimizer_G = torch.optim.Adam(
    itertools.chain(encoder.parameters(), decoder.parameters()), lr=0.0002, betas=(0.5, 0.999)
)

# reduce optimizer_D's lr 0.0002 to 0.0002/50
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002/50, betas=(0.5, 0.999))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [20]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits"""
    # Sample noise
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, 10))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "Exp03_images/%d.png" % batches_done, nrow=n_row, normalize=True)

## 04. Training

### Set parameters

* lr: learning rate
* 

In [23]:
for epoch in range(200):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        encoded_imgs = encoder(real_imgs)
        decoded_imgs = decoder(encoded_imgs)

        # Loss measures generator's ability to fool the discriminator
        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) + 0.999 * pixelwise_loss(
            decoded_imgs, real_imgs
        )

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as discriminator ground truth
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 10))))

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, 200, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % 400 == 0:
            sample_image(n_row=10, batches_done=batches_done)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[Epoch 23/200] [Batch 638/938] [D loss: 0.631937] [G loss: 0.067940]
[Epoch 23/200] [Batch 639/938] [D loss: 0.645175] [G loss: 0.073549]
[Epoch 23/200] [Batch 640/938] [D loss: 0.660817] [G loss: 0.078790]
[Epoch 23/200] [Batch 641/938] [D loss: 0.648555] [G loss: 0.083500]
[Epoch 23/200] [Batch 642/938] [D loss: 0.666250] [G loss: 0.078430]
[Epoch 23/200] [Batch 643/938] [D loss: 0.661747] [G loss: 0.074199]
[Epoch 23/200] [Batch 644/938] [D loss: 0.640669] [G loss: 0.077831]
[Epoch 23/200] [Batch 645/938] [D loss: 0.625773] [G loss: 0.079463]
[Epoch 23/200] [Batch 646/938] [D loss: 0.669482] [G loss: 0.074068]
[Epoch 23/200] [Batch 647/938] [D loss: 0.696463] [G loss: 0.063480]
[Epoch 23/200] [Batch 648/938] [D loss: 0.639407] [G loss: 0.075385]
[Epoch 23/200] [Batch 649/938] [D loss: 0.626762] [G loss: 0.075299]
[Epoch 23/200] [Batch 650/938] [D loss: 0.627017] [G loss: 0.071621]
[Epoch 23/200] [Batch 651/938] [D loss: 0.625611] [G loss: 0.078012

KeyboardInterrupt: ignored